In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import asf_search as asf
import logging

# asf.constants.INTERNAL.CMR_TIMEOUT = 90
# asf.ASF_LOGGER.setLevel(logging.DEBUG)

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
# results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]

args = asf.ASFSearchOptions(
    **{"start": '2022-02-10', "end": '2022-07-01'}
)
s = reference.stack(args)

### Create a Pair object from 2 products

In [ ]:
pair = asf.Pair(reference, s[2])

print(f"pair.ref.properties['sceneName']: {pair.ref.properties['sceneName']}")
print(f"pair.ref_date: {pair.ref_date}\n")

print(f"pair.sec.properties['sceneName']: {pair.sec.properties['sceneName']}")
print(f"pair.sec_date: {pair.sec_date}\n")

print(f"pair.temporal: {pair.temporal}")
print(f"pair.perpendicular: {pair.perpendicular}")

### Check the estimated coherence of the pair

In [ ]:
pair.estimate_s1_mean_coherence()

### Create a new pair with a long temporal baseline

In [ ]:
long_pair = pair = asf.Pair(reference, s[10])
print(f"long_pair.temporal: {long_pair.temporal}")

### Since the temporal baseline is greater than 48, an exception is raised when checking coherence

In [ ]:
long_pair.estimate_s1_mean_coherence()

### Demonstrate pair equivalence

Two pairs are equal if they both share the same reference and secondary dates.

In [ ]:
from copy import deepcopy

a = deepcopy(pair)
b = deepcopy(pair)
c = asf.Pair(reference, s[1])
print(f"a.ref_date: {a.ref_date}, a.sec_date: {a.sec_date}")
print(f"b.ref_date: {b.ref_date}, b.sec_date: {b.sec_date}")
print(f"c.ref_date: {c.ref_date}, c.sec_date: {c.sec_date}")
print(f"a == b: {a == b}")
print(f"a == c: {a == c}")

# Stack

### Create a Stack from the reference scene object instantiated above

Stack accepts ASFSearchOptions, which are used in a stack_from_product search

In [ ]:
args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-03-02'}
)

stack = asf.Stack(reference, opts=args)
stack.full_stack

### Remove pairs from the full stack, identifying them by ref and sec dates

Both full_stack and subset_stack contain the same Pair objects, so keeping both stacks does not eat up a ton of extra memory

In [ ]:
import numpy as np
import pandas as pd
import datetime

stack.remove_pairs([
    (datetime.date(2022, 1, 10), datetime.date(2022, 1, 22)), # datetime.date
    (pd.to_datetime("2022-1-10"), pd.to_datetime("2022- 2-3")), # pandas.Timestamp
    ("2022-01-10", "2022-02-15"), # isoformat string
    (np.datetime64("2022-01-10"), np.datetime64("2022-02-27")) # numpy.datetime64
])

stack.remove_list

### Look at subset_stack, after having removed some pairs from full_stack

In [ ]:
stack.subset_stack

### Look at connected_substacks

If subset_stack is disconnected, its component stacks are stored in connected_substacks.

Since we have not yet done anything to disconnect the subset_stack, this list contains only one item.

When we use the Network class to make SBAS stacks, we may have multiple connected substacks.



In [ ]:
stack.connected_substacks

## Network

Create a Network from the same reference scene. 

Attempts to create a connected, seasonal SBAS stack based on:
- georeference scene
- season
- start date
- end date
- perpendicular baseline
- temporal baseline
  

In [ ]:
import pandas as pd
from datetime import datetime

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)

season = ("1-1", "6-25")

opts = asf.ASFSearchOptions(
    **{
        "start": '2014-01-01', 
        "end": '2025-10-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    geo_reference = reference,
    perp_baseline=200, 
    inseason_temporal_baseline=12,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

In [ ]:
network.plot()

### Create an updated network with shorter baselines

In [ ]:
network = asf.Network(
    reference,
    perp_baseline=50, 
    inseason_temporal_baseline=12,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

network.plot()

### Create an updated network with longer baselines

In [ ]:
network = asf.Network(
    reference,
    perp_baseline=200, 
    inseason_temporal_baseline=24,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

network.plot()

### Plot full_stack

You can pass an any member stack (or list of member stacks) to the plot function.

In [ ]:
network.plot(stack_dict=network.full_stack)

### Call `Stack.get_insar_pair_ids()` on the largest connected substack

Provides the data structure you want when ordering GAMMA or single-burst interferograms from HyP3

In [ ]:
network.get_insar_pair_ids()

### `Stack.get_insar_pair_ids()` defaults to using the largest connected substack, but you can pass it any member stack

In [ ]:
print(len(network.get_insar_pair_ids()))
print(len(network.get_insar_pair_ids(network.full_stack)))

### MultiBurst Class

In [ ]:
# 3x3 adjactent bursts
# Should pass

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "173_370307": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

In [ ]:
# 3x3 adjactent bursts, but crosses orbital paths
# should fail

multiburst_dict = {
    "173_370305": ("IW1", "IW2", "IW3"),
    "173_370306": ("IW1", "IW2", "IW3"),
    "100_213509": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

In [ ]:
# Contains disconnected bursts and a hole
# Should fail

multiburst_dict = {
    "173_370305": ("IW1", "IW2",),
    "173_370306": ("IW1", "IW2", "IW3"),
    "173_370307": ("IW1", ),
    "173_370308": ( "IW3",),
    "173_370309": ("IW1", "IW2", "IW3"),
    "173_370310": ("IW1", "IW3"),
    "173_370311": ("IW1", "IW2", "IW3")
}

mb = asf.MultiBurst(multiburst_dict)

In [ ]:
# Diagonal line of bursts
# Should pass

multiburst_dict = {
    "173_370305": ("IW1",),
    "173_370306": ("IW2",),
    "173_370307": ("IW3",),
}

asf.MultiBurst(multiburst_dict)


In [ ]:
# "Vertical" zigzag line of bursts
# Should pass (but will currently fail when ordering from HyP3)

multiburst_dict = {
    "173_370305": ["IW1"],
    "173_370306": ["IW2"],
    "173_370307": ["IW1"],
}

asf.MultiBurst(multiburst_dict)

In [ ]:
# "Horizontal" zigzag line of bursts
# Should pass (but will currently fail when ordering from HyP3)

multiburst_dict = {
    "173_370305": ("IW1", "IW3"),
    "173_370306": ("IW2",)
}

asf.MultiBurst(multiburst_dict)


In [ ]:

# Intersects the Antimeridan
# Should fail

multiburst_dict = {
    "001_000664": ("IW1", "IW2", "IW3"),
    "001_000665": ("IW1", "IW2", "IW3"),
    "001_000666": ("IW1", "IW2", "IW3")
}

asf.MultiBurst(multiburst_dict)

### Multiburst Network

In [ ]:
multiburst_dict = {
    "085_181296": ("IW1", "IW2", "IW3"),
    "085_181297": ("IW1", "IW2", "IW3"),
    "085_181298": ("IW1", "IW2", "IW3")
}

multiburst_dict = {
    "085_181285": ("IW1", "IW2", ),
    "085_181286": ("IW1", "IW2", ),
    "085_181287": ("IW1", "IW2", "IW3" )
}

# # will generate CMR search error
# multiburst_dict = {
#     "093_197801": ("IW2", "IW3", ),
# }

multiburst = asf.MultiBurst(multiburst_dict)

In [ ]:
import asf_search as asf
import pandas as pd
from datetime import datetime

def get_julian_season(self) -> tuple[int,int]:
    season_start_ts = pd.Timestamp(
        datetime.strptime(f"{season[0]}-0001", "%m-%d-%Y"), tz="UTC"
        )
    season_start_day = season_start_ts.timetuple().tm_yday
    season_end_ts = pd.Timestamp(
        datetime.strptime(f"{season[1]}-0001", "%m-%d-%Y"), tz="UTC"
    )
    season_end_day = season_end_ts.timetuple().tm_yday
    return (season_start_day, season_end_day)

season = ("1-1", "6-25")

opts = asf.ASFSearchOptions(
    **{
        "start": '2015-01-01', 
        # "end": '2016-05-02', 
        # "start": '2022-01-01',
        "end": '2025-05-02', 
        "season": get_julian_season(season)
        }
)

network = asf.Network(
    multiburst = multiburst,
    perp_baseline=400, 
    inseason_temporal_baseline=48,
    bridge_target_date='3-1',
    bridge_year_threshold=2,
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

In [ ]:
network.plot()

In [ ]:
for n in network.additional_multiburst_networks:
    print(len(max(n.connected_substacks, key=len)))

# Force full connected stack

In [ ]:
#Creating network with multiple stacks

multiburst_dict = {
    "168_359019": ("IW3",),
    "168_359018": ("IW3",)
}
multiburst = asf.MultiBurst(multiburst_dict)

opts = asf.ASFSearchOptions(
    **{
        "start": '2016-01-01',
        "end": '2025-09-02', 
        }
)

network = asf.Network(
    multiburst = multiburst,
    perp_baseline=600, 
    inseason_temporal_baseline=60,
    bridge_target_date='9-1',
    bridge_year_threshold=1,
    opts=opts)

print(f"len(network.full_stack): {len(network.full_stack)}")
print(f"len(network.remove_list): {len(network.remove_list)}")
print(f"len(network.subset_stack): {len(network.subset_stack)}")
print(f"len(network.connected_substacks): {len(network.connected_substacks)}")

In [ ]:
network.plot()

## Connecting the stacks

In [ ]:
network.connect_components()

In [ ]:
network.plot()